In [1]:
#!apt-get update
!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#import sys
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install selenium
!pip install openpyxl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 33 not upgraded.
Need to get 75.5 MB of archives.
After this operation, 256 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 83.0.4103.61-0ubuntu0.18.04.1 [1,119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 83.0.410

In [2]:
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException,StaleElementReferenceException
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import _find_element, _find_elements
import openpyxl
import os
#import warnings
#warnings.filterwarnings('ignore')


In [3]:
class text_to_change(object):
    def __init__(self, locator, text):
        self.locator = locator
        self.text = text

    def __call__(self, driver):
        actual_text = _find_elements(driver, self.locator)[-1].text
        return actual_text != self.text
def init_driver():
  options = webdriver.ChromeOptions()
  options.headless = True
  #options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  prefs = {"profile.managed_default_content_settings.images": 2}
  options.add_experimental_option("prefs", prefs)
  options.add_experimental_option("excludeSwitches", ["enable-logging"])
  driver = webdriver.Chrome('chromedriver',options=options)
  return driver

In [4]:
"""#@title Default title text
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://www.google.com/maps/place/Travel+Club+Lounge/@22.6173602,88.3380781,12z/data=!4m8!1m2!2m1!1slounges+near+kolkata+airport!3m4!1s0x39f89e4cc784e89b:0x462b9adeee093971!8m2!3d22.6440875!4d88.4397318")
print(driver.page_source)  # results
# divs = wd.find_elements_by_css_selector('div')
"""

'#@title Default title text\noptions = webdriver.ChromeOptions()\noptions.add_argument(\'--headless\')\noptions.add_argument(\'--no-sandbox\')\noptions.add_argument(\'--disable-dev-shm-usage\')\n# open it, go to a website, and get results\ndriver = webdriver.Chrome(\'chromedriver\',options=options)\ndriver.get("https://www.google.com/maps/place/Travel+Club+Lounge/@22.6173602,88.3380781,12z/data=!4m8!1m2!2m1!1slounges+near+kolkata+airport!3m4!1s0x39f89e4cc784e89b:0x462b9adeee093971!8m2!3d22.6440875!4d88.4397318")\nprint(driver.page_source)  # results\n# divs = wd.find_elements_by_css_selector(\'div\')\n'

In [5]:
path = "/content/drive/My Drive/Colab Notebooks/Lounge_Air/Sent Codes/Review/Scraped Review Data/South America.xlsx"

In [6]:

wb = openpyxl.load_workbook(path)
sheet_obj = wb["input"]
m_row = sheet_obj.max_row
if "Output" not in wb.sheetnames:
    ws1 = wb.create_sheet("Output")
    wb["Output"].append(["Url", "Info1", "Info2", "Reviewer", "Oldness", "Review", "Stars","Country"])

inputlines = {}
for i in range(2, m_row + 1):
    status = sheet_obj.cell(row=i, column=6).value
    if not status:
        url = sheet_obj.cell(row=i, column=2).value
        info1 = sheet_obj.cell(row=i, column=4).value
        info2 = sheet_obj.cell(row=i, column=5).value
        country = sheet_obj.cell(row=i, column=7).value
        inputline = i
        if url:
            inputlines[url] = [info1, info2, inputline ,country]
        else:
            break

wb.save(path)
driver = init_driver()
try:
    for url in inputlines:
        driver.get(url)
        print(url)
        while True:
            try:
                WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='section-review-content']")))
                try:
                    load_all_reviews = driver.find_element_by_xpath("//div[@jsaction='pane.reviewlist.goToReviews']")
                    driver.execute_script("arguments[0].click();", load_all_reviews)
                    print("Loading reviews")
                except NoSuchElementException:
                    pass
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='section-loading-spinner']")))
                review_boxes = driver.find_elements_by_xpath("//div[@class='section-review-content']")
                last_review = review_boxes[-1].text

                loader = driver.find_element_by_xpath("//div[@class='section-loading-spinner']")
                driver.execute_script("arguments[0].scrollIntoView();", loader)
                print(f"Loaded: {len(review_boxes)} reviews")
                WebDriverWait(driver, 30).until(text_to_change((By.XPATH, "//div[@class='section-review-content']"), last_review))
            except TimeoutException:
                break

        print("No more reviews found. Scraping...")
        try:
          for review_box, index in zip(review_boxes, range(int(len(review_boxes)))):
              reviewer = review_box.find_element_by_xpath(".//div[@class='section-review-title']").text
              review_age = review_box.find_element_by_xpath(".//span[@class='section-review-publish-date']").text
              try:
                  show_more = review_box.find_element_by_xpath(".//button[@data-review-id and @ved and @jsaction and @class='section-expand-review blue-link']")
                  driver.execute_script("arguments[0].click();", show_more)
                  time.sleep(0.1)
              except NoSuchElementException:
                  pass
              review = review_box.find_element_by_xpath(".//span[@class='section-review-text']").text
              stars = review_box.find_element_by_xpath(".//span[@class='section-review-stars']").get_attribute("aria-label")
              wb["Output"].append([url, inputlines[url][0], inputlines[url][1], reviewer, review_age, review, stars,inputlines[url][3]])
              if index % 10 == 0:
                  print(f"Processed {index}/{len(review_boxes)} reviews")
          sheet_obj.cell(row=inputlines[url][2], column=6).value = "Scraping finished"
          wb.save(path)
        except NameError:
          sheet_obj.cell(row=inputlines[url][2], column=6).value = "Not found"
          wb.save(path)
          pass
        except StaleElementReferenceException:
          sheet_obj.cell(row=inputlines[url][2], column=6).value = "Not found"
          wb.save(path)
          pass

        print("Finished scraping. Saved to Excel")

finally:
    driver.quit()
